In [ ]:
from pymongo import MongoClient
import pymongo as pym
import seaborn as sns
from bson.objectid import ObjectId
from bson import Int64
%matplotlib inline

import matplotlib.pyplot as plt


from pylab import *
from scipy.optimize import curve_fit

import independent_cascade as ind_casc

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

from matplotlib.cbook import flatten

from scipy.optimize import minimize

import datetime
import pytz

from matplotlib.cbook import flatten

from scipy import stats

from sklearn import linear_model
from sklearn.metrics import f1_score, accuracy_score
import statistics

from sklearn import model_selection
import sklearn
from sklearn.cluster import DBSCAN, KMeans

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

import rake
import rake2
import operator
import string

QUERY_LIMIT = 60000
global generated_graph_components
generated_graph_components = None
import copy

In [ ]:
client = MongoClient()
db = client.faken
coll = db.bias_tweets_unique
cursor = db.bias_tweets_unique.find()

In [ ]:
cascade_representatives = list()

In [ ]:
def makeGraphDictFeatures(cursor, max_seconds=False):
    graph = nx.DiGraph()
    graph_dict = dict()
    for smth in cursor:
        orig_id = ''
        
        try: 
            orig_id = str(smth['retweeted_msg_id'])
            orig_id = orig_id.split('_')[-1]
            if max_seconds and smth['retweet_reaction_time_sec'] > max_seconds: # 5 minutes
                continue
            graph.add_edges_from([(orig_id, str(smth['msg_id']))])
        except KeyError: 
            graph.add_node(str(smth['msg_id']))
            
        
        
        el = smth
        try:
            supp=list(flatten(el["topics"]))
        except KeyError:
            supp = []
        if "TRUMP" in supp:
            el["TRUMP"]=1
        else:
            el["TRUMP"]=0
        if "HILARY" in supp:
            el["HILARY"]=1
        else:
            el["HILARY"]=0
        del el["topics"]
        graph_dict[str(smth['msg_id'])] = smth
        
    return (graph, graph_dict)

In [ ]:
def find_reshare_order(subgr, gr_dic):
    time_msg_dict = dict()
    for element in subgr.nodes():
        try:
            t = gr_dic[element]['retweet_reaction_time_sec']
        except KeyError:
            t = 0 # father
        try:
            time_msg_dict[t].append(element)
        except KeyError:
            time_msg_dict[t] = [element]
    
    ret = list()
    gr_dic[element]
    for k in sorted(time_msg_dict.keys()):
        ret.append((k,time_msg_dict[k]))
    return ret
    

In [ ]:
#####################
# Temporal Features #

def time_i(r_order, i): 
    try:
        return r_order[i][0]
    except IndexError:
        return 0

def time_first_half(r_order, k): 
    r_order = list(filter(lambda x: x[0] > 0, r_order)) # Do not count the 0-secs reshare of root
    max_index = int(k/2)-1
    lower_half_times = list()
    for x in r_order[0:max_index]:
        lower_half_times.append(x[0])
    deltalow = list()
    for i in range(len(lower_half_times)-1):
        deltalow.append(lower_half_times[i+1]-lower_half_times[i])
    if len(deltalow) > 0:
        return sum(deltalow)/int(len(deltalow))
    else:
        return 0

def time_second_half(r_order, k):
    r_order = list(filter(lambda x: x[0] > 0, r_order)) # Do not count the 0-secs reshare of root
    min_index = int(k/2)-1
    top_half_times = list()
    for x in r_order[min_index:int(k)-1]:
        if x[0] > 0: # Do not count the 0-secs reshare of root
            top_half_times.append(x[0])
    deltatop = list()
    for i in range(len(top_half_times)-1):
        deltatop.append(top_half_times[i+1]-top_half_times[i])
        
    if len(deltatop) > 0:
        return sum(deltatop)/int(len(deltatop))
    else:
        return 0
    

def time_change_reshares(r_order, k):
    or_order = list(r_order)#list(filter(lambda x: x[0] > 0, r_order))
    def f(x):
        res = 0
        t1 = 0
        tp2 = 0
        for i in range(or_order != [] and len(or_order)-1):
            t1 = time_i(or_order, i)
            tp1 = time_i(or_order, i+1) or t1
            res += ((tp1-t1)-(1+i)*x)**2
        return res
    x0 = 0.01
    return minimize(fun=f, x0=x0).x[0]



In [ ]:
def findComponents(graph, graph_dict):
    result = dict()
    father = ''
    for node in graph.nodes():
        try:
            

In [ ]:
def findAllChildren(cursor):
    ftutti_figli = list()
    print(datetime.datetime.now())
    print("Make children list")
    
    for ele in cursor:
        ftutti_figli.append(ele)

    print(datetime.datetime.now())
    return ftutti_figli

In [ ]:
def makeFamilies(children_list):
    print(datetime.datetime.now())
    print("Start making all families")

    ffather_son = dict()
    for f in children_list:
        try:
            ffather_son[f['retweeted_msg_id']].append(f)
        except KeyError:
            try:
                ffather_son[f['retweeted_msg_id']] = [f]
            except KeyError:
                ffather_son[f['msg_id']] = [f]

    print(datetime.datetime.now())
    return ffather_son

In [ ]:
def wienerIndexDictionary(graph):
    fwiener_total = dict()
    print("Starting Wiener index computations")
    print(datetime.datetime.now())
        
    for comp in nx.weakly_connected_component_subgraphs(graph):
        undir = nx.Graph()
        undir.add_edges_from(comp.edges())
        undir.add_nodes_from(comp.nodes())
        f_wien = nx.wiener_index(undir)
        for node in comp.nodes():
            fwiener_total[node] = f_wien
    print(datetime.datetime.now())
    return fwiener_total



In [ ]:
def addTimeWienerFeaturesPartial(graph, graph_dict, wiener_dict):
    wiener_median = statistics.median(wiener_dict.values())

    for comp in nx.weakly_connected_component_subgraphs(graph):
        res_order = find_reshare_order(comp, graph_dict)
        max_k= min(5, len(res_order[0][1]) - 1)
        tc = time_change_reshares(res_order, max_k)
        first = time_first_half(res_order, max_k)
        second = time_second_half(res_order, max_k)
        i_time = time_i(res_order, max_k)
        for node in comp.nodes():
            try:
                el = graph_dict[node] 
            except KeyError:
                continue
            el['tmp_feat_k'] = max_k
            el['tmp_feat_tk'] = i_time
            el['tmp_feat_firstk'] = first
            el['tmp_feat_secondk'] = second
            el['tmp_feat_time_chg'] = tc
            el['wiener_gt_median'] = int(wiener_dict[node]>wiener_median)
            el['wiener_index'] = wiener_dict[node]
            
            cascade_representatives.append(el)
            break

In [ ]:
def do_everything(list_cursors, list_isfake, cascade_representatives):
    
    for cur, flag in zip(list_cursors, list_isfake):
        print("Starting all!!!")
        print(datetime.datetime.now())
        generated_graph_components = None
        children = findAllChildren(cur)
        (generated_graph, generated_dict) = makeGraphDictFeatures(children)
        if generated_graph_components is None:
            generated_graph_components = nx.weakly_connected_component_subgraphs(generated_graph)
        wienerDict = wienerIndexDictionary(graph=generated_graph)
        addTimeWienerFeaturesPartial(generated_graph, generated_dict, wienerDict)
        print("All done!!")
        print(datetime.datetime.now())

        wiener_median = statistics.median(wienerDict.values())
        print(wiener_median)

        print('Wiener stats', 
              statistics.mean(wienerDict.values()), 
              statistics.mode(wienerDict.values()), 
              statistics.stdev(wienerDict.values()),
              statistics.median(wienerDict.values()),
              max(wienerDict.values()))
    def mapLocF(x):
        try:
            x['fake_news']
        except KeyError:
            x['fake_news'] = flag
        return x
    cascade_representatives = list(map(mapLocF, cascade_representatives))

In [ ]:
cascade_representatives = []
cursors = [db.fakes.find(), db.nonfakes.find().limit(QUERY_LIMIT)]
fake_flags = [1,0]
do_everything(cursors, fake_flags,cascade_representatives)

In [ ]:
# RUN ONCE!!
careasdasd() # dummy function to force you to read this! Run once only as it may overwrite an useful backup!
__cascade_complete_backup = copy.deepcopy(cascade_representatives)


In [ ]:
cascade_representatives = copy.deepcopy(__cascade_complete_backup)

In [ ]:
print(len(cascade_representatives))
print(statistics.mean(list(map(lambda x: x['wiener_index'], cascade_representatives))))

In [ ]:
cachedStopWords = stopwords.words("english")
lemmatiser = WordNetLemmatizer()

In [ ]:
completeText=''
s=''
for el in cascade_representatives:
    text = el['text']
    text = ' '.join([word for word in text.split() if word not in cachedStopWords and not word.startswith('#') and not word.startswith('@') and not word.startswith('http')])
    for word in text.split():
        s+=lemmatiser.lemmatize(word, pos="v")+' '
translator = str.maketrans('', '', string.punctuation)    
rake_object = rake.Rake('SmartStoplist.txt',3, 1, 15)
keywords = rake_object.run(s.translate(translator))
print ("Keywords:", keywords)


In [ ]:
for w in fn_words:
    if w in keywords_clean:
        print('a')

In [ ]:
keywords_clean = list(map(lambda x: x[0],keywords))
len(keywords_clean)

In [ ]:
len(cascade_representatives[0].keys())

In [ ]:
for el in cascade_representatives:
    for key in keywords_clean:
        if key in el['text']:
            el[str(key)]=1
        else:
            el[str(key)] = 0




In [ ]:

__cascade_backup =  copy.deepcopy(cascade_representatives)

In [ ]:
cascade_representatives = copy.deepcopy(__cascade_backup)

In [ ]:
y = list()
z = list()

In [ ]:

prohib_key = ['text', 'author', 'datetime', 'msg_id', 'is_manual_reply', 'is_manual_retweet', 
              'is_quote_retweet','is_reply_button', 'is_retweet_button', 'wiener_gt_median', 'wiener_index',
              'tmp_feat_k', 'tmp_feat_tk', 'tmp_feat_firstk', 'tmp_feat_secondk', 'tmp_feat_time_chg',
              'retweet_reaction_time_sec', 'retweeted_user', 'retweeted_user_followers_count', 'replied_user', 
              'retweeted_msg_id'
             ]
len(prohib_key)

In [ ]:
def shish(x):
    x['_id'] = str(x['_id'])
    return x
cascade_representatives = list(map(shish, cascade_representatives))

In [ ]:
for el in cascade_representatives:
    try:
        el['is_retweeted_user_verified']
    except KeyError:
        el['is_retweeted_user_verified'] = False
    for key in prohib_key:
        try:
            del el[key]
        except KeyError:
            continue
    y.append(el['fake_news'])
    z.append(el['_id'])
    del el['fake_news']
    del el['_id']
    
def create_sorted_list(x):
    ret = list()
    for k in sorted(x.keys()):
        ret.append(x[k])
    return ret

cascade_representatives = list(map(create_sorted_list, cascade_representatives))

In [ ]:
logreg = linear_model.LogisticRegression(n_jobs = 4)

In [ ]:
logreg.fit(cascade_representatives, y)

In [ ]:
id_proba = dict()
for theId,proba in zip(z, logreg.predict_proba(cascade_representatives)):
    id_proba[theId] = proba[1]
    

In [ ]:
tweet_ps = list()
for tw in __cascade_complete_backup:
    new = copy.deepcopy(tw)
    new['propensity_score'] = id_proba[str(new['_id'])]
    tweet_ps.append(new)

In [ ]:
#for tw in tweet_ps:
#    db.propensity_40.insert_one(tw)

In [ ]:
sns.set()
sns.distplot(list(map(lambda x: x['propensity_score'], tweet_ps)))

In [ ]:
sum(list(map(lambda x: int(x['propensity_score']<0.3), tweet_ps)))

In [ ]:
bins = dict()
N_bins = 20
for el in tweet_ps:
    
    for prop in np.linspace(0,1,N_bins):
        if el['propensity_score'] < prop:
            try:
                bins[prop].append(el)
            except KeyError:
                bins[prop] = [el]
            break

In [ ]:
for l in sorted(bins.keys()):
    print(l,len(bins[l]))

In [ ]:
fake_w = dict()
non_fake_w = dict()
c=0
for k in bins.keys():
    fake_w[k] = []
    non_fake_w[k] = []
    for el in bins[k]:
        if el['fake_news']:
            fake_w[k].append(el['wiener_index'])
        else:
            non_fake_w[k].append(el['wiener_index'])
            
for k in sorted(bins.keys()):
    print(k, "Fake: ", statistics.mean(fake_w[k]), "#fake: ", len(fake_w[k]))
    print(k, "NonF: ", statistics.mean(non_fake_w[k]), len(non_fake_w[k]), end='\n\n')
    

In [ ]:
for k in bins.keys():
    if not str(k).startswith('0.15'):
        continue
   
    for el in bins[k]:
        if el['fake_news']:
            print(el)
            break
            

In [ ]:
fa = list()
nf = list()
for k in bins.keys():
    fa.append(statistics.mean(fake_w[k]))
    nf.append(statistics.mean(non_fake_w[k]))
print("Fake:", statistics.mean(fa))
print("NFake:", statistics.mean(nf))


In [ ]:
fak = 0
nfak = 0
for k in bins.keys():
    if statistics.mean(fake_w[k]) > statistics.mean(non_fake_w[k]):
        fak+=1
    else:
        nfak +=1
        
print(fak, nfak)

In [ ]:
logreg2 = linear_model.LogisticRegression(n_jobs = 4)
thr = int(2*len(cascade_representatives)/3)
x_train = cascade_representatives[:thr]
x_test = cascade_representatives[thr:]
y_train = y[:thr]
y_test = y[thr:]

In [ ]:
logreg2.fit(x_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(logreg2, cascade_representatives, y, cv=10)
statistics.mean(scores)



In [ ]:
statistics.mean(scores), statistics.stdev(scores)

In [ ]:
fake_tweets = dict()
non_fake_tweets = dict()
c=0
for k in bins.keys():
    fake_tweets[k] = []
    non_fake_tweets[k] = []
    for el in bins[k]:
        print(el)
        break
        if el['fake_news']:
            fake_tweets[k].append(el)
        else:
            non_fake_tweets[k].append(el)
      

In [ ]:
fake_tweets = dict()
non_fake_tweets = dict()
c=0
for k in bins.keys():
    fake_tweets[k] = []
    non_fake_tweets[k] = []
    for el in bins[k]:
        try:
            if el['fake_news']:
                fake_tweets[k].append(el)
            else:
                non_fake_tweets[k].append(el)
        except KeyError:
            non_fake_tweets[k].append(el)
            